In [1]:
# General Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import defaultdict
import pickle
from PIL import Image
import urllib
from adjustText import adjust_text


In [2]:
chromedriver_autoinstaller.install()

import warnings
warnings.filterwarnings('ignore')

## Pull Data

In [3]:
chromedriver_autoinstaller.install()

if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640883/Live/England-Premier-League-2022-2023-Chelsea-Leeds"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

Region: England, League: Premier League, Season: 2022/2023, Match Id: 1640883


In [4]:
homeID = home_data['teamId']
awayID = away_data['teamId']

names = {
    homeID : home_name, 
    awayID : away_name
}

## Colours 

In [5]:
home_color = '#2D5DE7'
away_color = '#9C9C9C'

## Ground Duels Won
- Includes situations where two players involved for a battle of the ball
- Ground Duels Won = Succ Dribbles + Succ Tackles + Fouls Won

In [42]:
ground_duels = events_df[(events_df['type'].isin(['Foul', 'TakeOn', 'Tackle'])) & 
                         (events_df['outcomeType'] == 'Successful')]

# Home Team
ground_duels_home = ground_duels[ground_duels['teamId'] == homeID].reset_index(drop=True)
ground_duels_home = (pd.DataFrame(ground_duels_home.groupby('playerName')['type']
                                                   .count()
                                                   .reset_index())
                    )
ground_duels_home.rename({'type':'ground_duels'}, axis='columns', inplace=True)
ground_duels_home


# Away Team 
ground_duels_away = ground_duels[ground_duels['teamId'] == awayID].reset_index(drop=True)
ground_duels_away = (pd.DataFrame(ground_duels_away.groupby('playerName')['type']
                                                   .count()
                                                   .reset_index())
                    )
ground_duels_away.rename({'type':'ground_duels'}, axis='columns', inplace=True)
ground_duels_away

,playerName,ground_duels
0,Brenden Aaronson,2
1,Crysencio Summerville,10
2,Georginio Rutter,4
3,Jack Harrison,4
4,Junior Firpo,5
5,Luke Ayling,2
6,Maximilian Wöber,2
7,Robin Koch,1
8,Tyler Adams,3
9,Weston McKennie,4


## Ground Duels Won

In [57]:
aerial_duels = events_df[(events_df['type'] == 'Aerial') & 
                         (events_df['outcomeType'] == 'Successful')]

# Home Team
aerial_duels_home = aerial_duels[aerial_duels['teamId'] == homeID].reset_index(drop=True)
aerial_duels_home = (pd.DataFrame(aerial_duels_home.groupby('playerName')['type']
                                                 .count()
                                                 .reset_index())
                    )
aerial_duels_home.rename({'type':'aerial_duels'}, axis='columns', inplace=True)
aerial_duels_home


# Away Team 
aerial_duels_away = aerial_duels[aerial_duels['teamId'] == awayID].reset_index(drop=True)
aerial_duels_away = (pd.DataFrame(aerial_duels_away.groupby('playerName')['type']
                                                   .count()
                                                   .reset_index())
                    )
aerial_duels_away.rename({'type':'aerial_duels'}, axis='columns', inplace=True)
aerial_duels_away

,playerName,aerial_duels
0,Georginio Rutter,1
1,Junior Firpo,2
2,Luke Ayling,1
3,Maximilian Wöber,1
4,Robin Koch,5
5,Tyler Adams,3
6,Weston McKennie,3


## Merge Datasets

In [62]:
# Home Team
df_home = (
            ground_duels_home.merge(aerial_duels_home, how='outer', on='playerName')
                             .fillna(0)
)

# Away Tea,
df_away = (
            ground_duels_away.merge(aerial_duels_away, how='outer', on='playerName')
                             .fillna(0)
)
df_away

,playerName,ground_duels,aerial_duels
0,Brenden Aaronson,2,0.0
1,Crysencio Summerville,10,0.0
2,Georginio Rutter,4,1.0
3,Jack Harrison,4,0.0
4,Junior Firpo,5,2.0
5,Luke Ayling,2,1.0
6,Maximilian Wöber,2,1.0
7,Robin Koch,1,5.0
8,Tyler Adams,3,3.0
9,Weston McKennie,4,3.0


## Plot 

In [71]:
fig, ax = plt.subplots(figsize = (20,15))
fig.set_facecolor("#2B2B2B")
ax.patch.set_facecolor("#2B2B2B")
mpl.rcParams['figure.dpi'] = 800


# Home Team
plt.scatter(df_home.aerial_duels, df_home.ground_duels, color=home_color, s=40)

text = []
for i, row in df_home.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['aerial_duels']
    y = row['ground_duels']
    sf = float(x+y)
    t = plt.text(x = x+0.0005, y = y+0.002, s = p1.upper(), 
                 bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(14, min(40, 5.5*(x+y))))
    text.append(t)
    

# Away Team
plt.scatter(df_away.aerial_duels, df_away.ground_duels, color=away_color, s=40)

for i, row in df_away.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['aerial_duels']
    y = row['ground_duels']
    sf = float(x+y)
    t = plt.text(x = x+0.0005, y = y+0.002, s = p1.upper(), 
                 bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(14, min(40, 5.5*(x+y))))
    text.append(t)
    
plt.title('\nDuels Won \n\n'.upper(), 
          loc='left', color='w', fontname = 'Sans Serif', fontsize = 30, y=1)
plt.suptitle(t = home_name.upper() + ' ' +score + ' ' + away_name.upper() + '  |  ' + date_str.upper() + '  |  '  + 'CREATED BY ...', x=0.125, y=0.93, ha='left', 
             fontsize=24, color='w')
plt.xlabel('\nAerial Duels Won', color='w', fontsize=30)
plt.ylabel('Ground Duels Won\n', color='w', fontsize=30)

ax.set_axisbelow(True)
ax.yaxis.grid(color='#848484', linestyle='dashed')
ax.xaxis.grid(color='#848484', linestyle='dashed')
# spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('w')
ax.spines['bottom'].set_color('w')
# ticks
for tick in ax.xaxis.get_ticklines(): 
    tick.set_color('w')
for tick in ax.yaxis.get_ticklines(): 
    tick.set_color('w')
plt.xticks(color='w')
plt.yticks(color='w')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

adjust_text(text, ax=ax)

plt.figtext(0.125, 0.005, "> Ground duels includes successful tackles, dribbles and fouls won", ha="left", fontsize=16, fontname='Sans Serif', color='white')



fig.savefig('duels_scatter.png', 
            bbox_inches="tight",
            edgecolor="none",
            facecolor = "#2B2B2B",
            dpi=500)